# Billards & Collisions

Simulation of a ball bouncing inside a circular boundary using event detection.

You can also find this example as a standalone Python file in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/examples/examples_event/example_billards_sphere.py).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style
plt.style.use('../pathsim_docs.mplstyle')

from pathsim import Simulation, Connection
from pathsim.blocks import Constant, Integrator, Scope
from pathsim.events import ZeroCrossingUp
from pathsim.solvers import RKBS32

In [ ]:
# System parameters
g = 9.81   # gravity [m/s^2]
l = 1      # radius of circular boundary [m]

# Initial conditions
x0 = np.array([0.5, 0.5])  # initial position [m]
v0 = np.array([0, 0])      # initial velocity [m/s]

In [ ]:
# Blocks for dynamics
cn = Constant(-g)                    # gravitational acceleration
ix = Integrator(x0)                  # position integrator
iv = Integrator(v0)                  # velocity integrator
sc = Scope(labels=["x", "y"])        # scope to record position

## Collision Detection and Response

The collision is detected using a zero-crossing event. The detection function computes the distance from the origin minus the boundary radius:

$$
f(x) = \|x\| - l
$$

When this function crosses zero from below (the ball reaches the boundary), the action function is triggered. The elastic reflection is computed as:

$$
v_{\text{new}} = v - 2(v \cdot n)n
$$

where $n = x / \|x\|$ is the outward normal at the collision point.

In [ ]:
# Collision event functions
def bounce_detect(_):
    """Detect when ball reaches boundary."""
    x = ix.engine.get()
    return np.linalg.norm(x) - l 

def bounce_act(_):
    """Reflect velocity elastically off boundary."""
    v = iv.engine.get()
    x = ix.engine.get()
    n = x / np.linalg.norm(x)  # outward normal
    iv.engine.set(v - 2 * np.dot(v, n) * n)  # reflect velocity
    ix.engine.set(l * n)  # ensure position is exactly on boundary

In [ ]:
# Simulation definition
sim = Simulation(
    blocks=[ix, iv, sc, cn],
    connections=[
        Connection(cn, iv[1]),         # gravity -> velocity y-component
        Connection(iv[0,1], ix[0,1]),  # velocity -> position
        Connection(ix[0,1], sc[0,1]),  # position -> scope
    ],
    events=[
        ZeroCrossingUp(
            func_evt=bounce_detect, 
            func_act=bounce_act,
        ),
    ],
    Solver=RKBS32,
    dt_max=0.01
)

In [ ]:
# Run the simulation
sim.run(7)

Let's visualize the results. First, we plot the x and y coordinates over time:

In [ ]:
# Plot position components over time
sc.plot();

The 2D trajectory shows the ball bouncing inside the circular boundary. We overlay the boundary circle for reference:

In [ ]:
# Plot 2D trajectory with boundary
t, (x, y) = sc.read()

fig, ax = plt.subplots()
ax.plot(x, y)

# Draw circular boundary
ang = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(ang), np.sin(ang), color="grey", linewidth=2)
ax.set_aspect('equal')
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]");